## Data cleaning lesson/exercise from [DataQuest](https://www.dataquest.io/m/136/data-cleaning-walkthrough)

Read CSV datasets into Pandas DataFrame objects that we'll store in a dictionary

In [1]:
import pandas as pd

# NYC high school datasets we'll use, from NYC Open Data (https://data.cityofnewyork.us/Education)
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

# create a dictionary of data file names, minus .csv extension (keys) to corresponding Pandas DataFrames (values) 
data = {}
for data_file in data_files:
    dataset_name = data_file.split(".")[0]
    data[dataset_name] = pd.read_csv(data_file)

Read survey datasets which are not CSV but instead are tab delimited text files with a "windows-1252" encoding:

In [3]:
# read the survey datasets into Pandas DataFrame objects
all_survey = pd.read_csv("survey_all.txt", 
                         delimiter="\t", 
                         encoding="windows-1252")
d75_survey = pd.read_csv("survey_d75.txt", 
                         delimiter="\t", 
                         encoding="windows-1252")

# combine the two surveys into a single DataFrame object
survey = pd.concat((all_survey, d75_survey), axis=0, sort=True)

C:\home\miniconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  import sys


Clean the survey DataFrame by 

1. renaming the "dbn" field to "DBN", and 
2. filtering down to only the fields we'll need for our analysis

In [4]:
# copy the "dbn" column as a column named "DBN"
survey["DBN"] = survey["dbn"]

# create a list of the relevant fields we'll want to filter into our "survey" DataFrame
survey_fields = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]

# filter the DataFrame, assign it into the data dictionary
data["survey"] = survey[survey_fields]

Create a function to help with creating a DBN column for the class_size DataFrame. The function should behave as follows:

- Takes in a number.
- Converts the number to a string using the str() function.
- Pads the string with a zero if the length of the string is only 1 digit.
- Returns the string

In [7]:
def num_to_str(number):
    return str(number).zfill(2)

Create a DBN column in the class_size DataFrame from the "CSD" (padded with a leading zero if necessary) and "SCHOOL CODE" columns.

In [8]:
data["class_size"]["DBN"] = data["class_size"]["CSD"].apply(lambda x: num_to_str(x)) + data["class_size"]["SCHOOL CODE"]

Create a "DBN" column in the "hs_directory" DatFrame (copy the "dbn" column and name it "DBN").

In [10]:
data["hs_directory"].rename(columns={'dbn': 'DBN'}, inplace=True)

Create a total SAT score column from the three component SAT score columns in the `sat_results` DataFrame.

In [11]:
cols = ["SAT Math Avg. Score", 
        "SAT Critical Reading Avg. Score", 
        "SAT Writing Avg. Score"]
for col in cols:
    data["sat_results"][col] = \
        pd.to_numeric(data["sat_results"][col], errors="coerce")

data["sat_results"]["sat_score"] = \
    data["sat_results"]["SAT Math Avg. Score"] +\
    data["sat_results"]["SAT Critical Reading Avg. Score"] +\
    data["sat_results"]["SAT Writing Avg. Score"]

Use a regular expression to pull the lat/lon values from the `Location 1` column of the `hs_directory` DataFrame:

In [29]:
import re

def extract_lat(location):
    
    # get the first part of the lat/lon pair
    lat = re.findall("\(.+\)", location)[0].split()[0]
    
    # clean off the leading parenthesis and the comma
    lat = lat.replace("(", "").replace(",", "")
    
    # return the lat value as a string
    return lat
    
    
def extract_lon(location):
    
    # get the second part of the lat/lon pair
    lon = re.findall("\(.+\)", location)[0].split()[1]
    
    # clean off the trailing parenthesis and the comma
    lon = lon.replace(")", "").replace(",", "")
    
    # return the lon value as a string
    return lon
    
# create lat and lon columns from the lat/lon values extracted from the "Location 1" column
data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(extract_lat) 
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(extract_lon) 

# convert the string values to numeric
data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"],
                                            errors="coerce")
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"],
                                            errors="coerce")

# look to make sure we have what we expect
data["hs_directory"][["Location 1", "lat", "lon"]].head()

,Location 1,lat,lon
0,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",40.670299,-73.961648
1,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",40.827603,-73.904475
2,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",40.842414,-73.916162
3,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",40.710679,-74.000807
4,"160 20 Goethals Avenue\nJamaica, NY 11432\n(40...",40.718810,-73.806500


## Condense the `class_size` DataFrame

First we'll filter the `class_size` DataFrame down to general education classes containing grades 9-12:

In [30]:
class_size = data["class_size"]
data["class_size"] = class_size[(class_size["GRADE "] == "09-12") & (class_size["PROGRAM TYPE"] == "GEN ED")]
data["class_size"].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
225,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01M292
226,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01M292
227,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01M292
228,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 12,-,69.0,3.0,23.0,13.0,30.0,STARS,NaN,01M292
229,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Integrated Algebra,-,53.0,3.0,17.7,16.0,21.0,STARS,NaN,01M292


Combine the various class sizes for non-unique `DBN` rows into unique rows for each `DBN` using an aggregation of mean values with a groupby object:

In [32]:
import numpy as np
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
data["class_size"] = class_size
data["class_size"].head()

,DBN,CSD,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,SCHOOLWIDE PUPIL-TEACHER RATIO
0,01M015,1,17.727273,1.090909,16.545455,16.454545,16.636364,8.4
1,01M019,1,24.250000,1.333333,17.625000,17.333333,17.916667,10.0
2,01M020,1,43.923077,2.076923,21.976923,21.307692,22.538462,13.4
3,01M034,1,38.038462,1.692308,21.461538,20.730769,22.038462,9.6
4,01M063,1,16.400000,1.000000,16.400000,16.400000,16.400000,7.6


### Condense the `demographics` DataFrame down to the school year for which we have our SAT dataset: 

In [33]:
demographics = data["demographics"]
demographics = demographics[demographics["schoolyear"] == 20112012]
data["demographics"] = demographics
data["demographics"].head()

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,...,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,...,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3
27,01M034,PS 034 FRANKLIN D ROOSEVELT,20112012,NaN,99.7,401,14,34,38,36,...,90,22.4,275,68.6,8,2.0,204.0,50.9,197.0,49.1
35,01M063,PS 063 WILLIAM MCKINLEY,20112012,NaN,78.9,176,18,20,30,21,...,41,23.3,110,62.5,15,8.5,97.0,55.1,79.0,44.9


### Condense the `graduation` DataFrame down to the actual/total cohort groups, in order to assign each a unique `DBN` identifier.

In [34]:
graduation = data["graduation"]
graduation = graduation[(graduation["Cohort"] == "2006") & (graduation["Demographic"] == "Total Cohort")]
data["graduation"] = graduation
data["graduation"].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,...,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
10,Total Cohort,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,124,53,42.7%,42,33.9%,79.2%,...,34,27.4%,64.2%,11,8.9%,20.8%,46,37.1%,20,16.100000000000001%
17,Total Cohort,01M450,EAST SIDE COMMUNITY SCHOOL,2006,90,70,77.8%,67,74.400000000000006%,95.7%,...,67,74.400000000000006%,95.7%,3,3.3%,4.3%,15,16.7%,5,5.6%
24,Total Cohort,01M509,MARTA VALLE HIGH SCHOOL,2006,84,47,56%,40,47.6%,85.1%,...,23,27.4%,48.9%,7,8.300000000000001%,14.9%,25,29.8%,5,6%
31,Total Cohort,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHO,2006,193,105,54.4%,91,47.2%,86.7%,...,22,11.4%,21%,14,7.3%,13.3%,53,27.5%,35,18.100000000000001%


#### Since we'll be looking at AP students to see how they relate to SAT scores we'll want to have a dataset containing values related to AP tests. We'll first make sure that the relevant columns are of numeric type:

In [35]:
cols = ['AP Test Takers ', 
        'Total Exams Taken', 
        'Number of Exams with scores 3 4 or 5']

ap_2010 = data["ap_2010"]
for col in cols:
    ap_2010[col] = pd.to_numeric(ap_2010[col], errors='coerce')
data["ap_2010"] = ap_2010